In [41]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU

In [42]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

C:\Users\Akhlaq\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\client\session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [43]:
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [44]:
#Resizing images, if needed
SIZE_X = 256 
SIZE_Y = 256
n_classes = 6

In [45]:
train_images_org = []

d = r"C:\Users\Akhlaq\PythonProject\trainingData_semanticSegmentation\trainingData_semanticSegmentation\0_original_images"

for directory_path in glob.glob(d):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 1)
        img = (img/255.0).astype(np.float32)
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images_org.append(img)

In [46]:
train_images = np.array(train_images_org, dtype=np.float32)

In [47]:
masks = [] 

d = r"C:\Users\Akhlaq\PythonProject\Adjusted_Masks"

for directory_path in glob.glob(d):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        mask = mask.astype(np.int32)
        # mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        masks.append(mask)

In [48]:
train_masks = np.array(masks, dtype=np.int32)

In [49]:
train_images.shape

(474, 256, 256, 3)

In [50]:
train_masks.shape

(474, 256, 256)

In [51]:
np.unique(train_masks)

array([0, 1, 2, 3, 4, 5])

In [52]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
h, w, c = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(h, w, c)

In [53]:
np.unique(train_masks_encoded_original_shape)

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [54]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [55]:
train_masks_input.shape

(474, 256, 256, 1)

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

In [57]:
from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

In [58]:
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))
test_masks_cat.shape

(48, 256, 256, 6)

In [59]:
n_classes=6
activation='softmax'

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)

In [60]:
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [61]:
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
# model1.compile(optim, total_loss, metrics=metrics)

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

print(model1.summary())


history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=0,
          validation_data=(X_test1, y_test_cat))

model1.save('res34_backbone_50epochs.hdf5')

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, None, None,  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_34 (ZeroPadding2 (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_34[0][0]          
_______________________________________________________________________________________

In [ ]:
from tensorflow.keras.models import load_model

### FOR NOW LET US FOCUS ON A SINGLE MODEL

#Set compile=False as we are not loading it for training, only for prediction.
model1 = load_model('res34_backbone_50epochs.hdf5', compile=False)

#IOU
y_pred1=model1.predict(X_test)
y_pred1_argmax=np.argmax(y_pred1, axis=3)


#Using built in keras function
#from keras.metrics import MeanIoU
n_classes = 6
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred1_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
i = 0
class0_IoU = values[i,i]/(values[i,i] + values[i,1] + values[i,2] + values[i,3] + values[i,4] + values[i,5] + values[1,i]+ values[2,i]+ values[3,i] + values[4,i] + values[5,i])
i = 1
class1_IoU = values[i,i]/(values[i,i] + values[i,0] + values[i,2] + values[i,3] + values[i,4] + values[i,5] + values[0,i]+ values[2,i]+ values[3,i] + values[4,i] + values[5,i])
i = 2
class2_IoU = values[i,i]/(values[i,i] + values[i,0] + values[i,1] + values[i,3] + values[i,4] + values[i,5] + values[0,i]+ values[1,i]+ values[3,i] + values[4,i] + values[5,i])
i = 3
class3_IoU = values[i,i]/(values[i,i] + values[i,0] + values[i,1] + values[i,2] + values[i,4] + values[i,5] + values[0,i]+ values[1,i]+ values[2,i] + values[4,i] + values[5,i])
i = 4
class4_IoU = values[i,i]/(values[i,i] + values[i,0] + values[i,1] + values[i,2] + values[i,3] + values[i,5] + values[0,i]+ values[1,i]+ values[2,i] + values[3,i] + values[5,i])
i = 5
class5_IoU = values[i,i]/(values[i,i] + values[i,0] + values[i,1] + values[i,2] + values[i,3] + values[i,4] + values[0,i]+ values[1,i]+ values[2,i] + values[3,i] + values[4,i])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)
print("IoU for class4 is: ", class5_IoU)
print("IoU for class4 is: ", class6_IoU)

In [ ]:
plt.imshow(train_images[4, :,:,0], cmap='gray')
plt.imshow(train_masks[4], cmap='jet')

In [ ]:
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
import random
test_img_number = random.randint(0, len(X_test1))
test_img = X_test1[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)

test_img_input1 = preprocess_input1(test_img_input)

test_pred1 = model1.predict(test_img_input1)
test_prediction1 = np.argmax(test_pred1, axis=3)[0,:,:]




plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0])
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction1, cmap='gray')
plt.show()